<a href="https://colab.research.google.com/github/Ssnowzx/Aula01_agentesAI-Alura/blob/main/Imers%C3%A3o_Dev_Agentes_de_IA_ALURA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AULA 01


In [2]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.1 MB/s eta 0:00:00


IMPORTACAO DA API_KEY

In [11]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY = userdata.get('GEMINAI_API_KEY')

CHAMADA COM O GEMINAI

In [21]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [ ]:
resp_test = llm.invoke("quem é vc? seja criativo")
print(resp_test.content)

In [51]:
TRIAGEM_PROMPT = (
    "Você é um classificador de consultas para o Suporte ao Cliente da empresa ShopNova ADRENA E-commerce. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre procedimentos padrão de compras ou entregas (Ex: "Onde está meu pedido?", "Como faço uma devolução simples?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Tenho um problema com minha compra", "Não sei o que aconteceu").\n'
    '- **ABRIR_CHAMADO**: Pedidos de reembolso especial, cancelamento urgente, queixas ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero reembolso imediato por produto defeituoso.", "O delivery atrasou, exijo compensação.", "Por favor, abra um chamado para reclamações.").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [23]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class TriagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER", "PEDIR_INFO", "ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA", "MEDIA", "ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [34]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY
)

In [44]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(TriagemOut)

def triagem(mensagem: str) -> Dict:
    saida: TriagemOut = triagem_chain.invoke([
        SystemMessage(content=TRIAGEM_PROMPT),
        HumanMessage(content=mensagem)
    ])

    return saida.model_dump()

In [52]:
testes = ["Onde está meu pedido?",
          "Quero reembolso imediato por produto defeituoso. Como faço?",
          "Como faço uma devolução?",
          "Quantos produtos tem na loja?"]

In [53]:
for msg_teste in testes:
    print(f"Pergunta: {msg_teste}\n -> Resposta: {triagem(msg_teste)}\n")

Pergunta: Onde está meu pedido?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quero reembolso imediato por produto defeituoso. Como faço?
 -> Resposta: {'decisao': 'ABRIR_CHAMADO', 'urgencia': 'ALTA', 'campos_faltantes': []}

Pergunta: Como faço uma devolução?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

Pergunta: Quantos produtos tem na loja?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

